In [1]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs

In [2]:
dpau0 = hs.load("Data/AuTwinTilt/20220309_133028_p_0/dataset-2t.hspy")

ERROR:numba.cuda.cudadrv.driver:Call to cuInit results in UNKNOWN_CUDA_ERROR


In [3]:
dpau0.plot(vmax = 30, cmap = 'inferno_r', scalebar = False)

In [14]:
dpau0.center_direct_beam(method="blur",half_square_width = 20, sigma = 3)

[########################################] | 100% Completed |  7.8s


In [3]:
from skimage import filters

def crop_minimum(image, minimum=3):
    copied = image.copy()
    copied[copied <= minimum] = 0.
    return copied

In [9]:
temp = dpau0.inav[50,90].subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=7,
                                                            max_sigma=10, 
                                                            lazy_result=False)
temp.map(filters.gaussian, sigma=1)
temp.map(crop_minimum, minimum = 1)
temp.map(filters.gaussian, sigma=0.5)

[########################################] | 100% Completed |  0.1s


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
procdat.plot(vmax = 30)

In [11]:
dpau0 = dpau0.subtract_diffraction_background(method="difference of gaussians",
                                                            min_sigma=7,
                                                            max_sigma=10, 
                                                            lazy_result=True)
dpau0.map(filters.gaussian, sigma=1)
dpau0.map(crop_minimum, minimum = 1)
dpau0.map(filters.gaussian, sigma=0.5)

<LazyDiffraction2D, title: , dimensions: (128, 128|256, 256)>

In [12]:
dpau0.compute()

[########################################] | 100% Completed | 50.5s


In [10]:
test_patterns = hs.signals.Signal2D((dpau.inav[40,73].data,
                                     dpau.inav[84,87].data,
                                     dpau.inav[36,72].data,
                                     dpau.inav[111,96].data,
                                    ))

In [13]:
dpau0.plot(vmax = 30)

In [16]:
# dpau0.save('Tilt4bgred.hspy')

In [108]:
dpau = hs.load('Tilt1bgred.hspy')

In [3]:
dpau.plot(vmax = 30,cmap = 'viridis')

In [2]:
import diffpy
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from diffsims.libraries.diffraction_library import DiffractionLibrary
from pyxem.utils.calibration_utils import find_diffraction_calibration

In [64]:
%%capture burn
import diffpy
def template_log_func(x):
    return np.log10(x + 0.001)

resolution = 1 
grid = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
structure_matrix = diffpy.structure.loadStructure("Data/Au_mp-81_conventional_standard.cif")

diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)
library_phases = StructureLibrary(["phase"], [structure_matrix], [grid])

found_cal, corrline, found_cals = find_diffraction_calibration(test_patterns,0.0093,library_phases,lib_gen,20,
                                                               max_excitation_error= 0.08,
                                                               intensity_transform_function=template_log_func,
                                                               delta_r = 0.5,
                                                               max_r = np.sqrt(128**2+128**2) * 0.90
                                                              )

In [65]:
found_cals

array([0.0093    , 0.00916634, 0.008835  , 0.00916634])

In [4]:
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
resolution = 0.5 # maximum angle in degrees between nearest templates. Pretty rough grid for speed.
grid_cub = get_beam_directions_grid("cubic", resolution, mesh="spherified_cube_edge")
print("Number of patterns: ", grid_cub.shape[0])

Number of patterns:  4186


/home/joseph/miniconda3/envs/pyxemmaster/lib/python3.9/site-packages/diffsims/generators/sphere_mesh_generators.py:515: RuntimeWarning: invalid value encountered in true_divide
  phi2 = sign * np.nan_to_num(np.arccos(x_comp / norm_proj))


In [5]:
import diffpy
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator

In [6]:
# Parameters necessary for simulating a template library
diffraction_calibration = 0.0091605 
# half size of the images
half_shape = (dpau.data.shape[-2]//2, dpau.data.shape[-1]//2)
# maximum radius in reciprocal space to calculate spot intensities for
reciprocal_radius = np.sqrt(half_shape[0]**2 + half_shape[1]**2)*diffraction_calibration
reciprocal_radius

1.6582276272791985

In [7]:
# importing the structures
# structure_matrix = diffpy.structure.loadStructure("Data/Cu_mp-30_conventional_standard.cif")
# structure_matrix = diffpy.structure.loadStructure("Data/Si_mp-149_conventional_standard.cif")
structure_matrix = diffpy.structure.loadStructure("Data/Au_mp-81_conventional_standard.cif")
# "The microscope"
grid = get_beam_directions_grid("cubic", 0.5, mesh="spherified_cube_edge")
diff_gen = DiffractionGenerator(accelerating_voltage=200,
                                precession_angle=1,
                                scattering_params=None,
                                shape_factor_model="linear",
                                minimum_intensity=0.1,
                                )

lib_gen = DiffractionLibraryGenerator(diff_gen)

# Generating a library
# "Library of structures and orientations"
library_phases = StructureLibrary(["Au"], [structure_matrix], [grid])
# Calculate the actual library

diff_lib = lib_gen.get_diffraction_library(library_phases,
                                           calibration=diffraction_calibration,
                                           reciprocal_radius=reciprocal_radius,
                                           half_shape=half_shape,
                                           with_direct_beam=False,
                                           max_excitation_error=0.1)

In [8]:
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls

In [10]:
simulations = diff_lib["Au"]["simulations"]
image = dpau.inav[50,90].data
frac_keep = 1
n_keep = None
n_best = 1
delta_r = 0.5
delta_theta = 0.5
max_r = None
intensity_transform_function = template_log_func
find_direct_beam = False
direct_beam_position = None
normalize_image = True
normalize_templates = True
indices_n, angles_n, correlations_n, signs_n = iutls.get_n_best_matches(image,
                                                 simulations,
                                                 n_best,
                                                 frac_keep,
                                                 n_keep,
                                                 delta_r,
                                                 delta_theta,
                                                 max_r,
                                                 intensity_transform_function,
                                                 find_direct_beam,
                                                 direct_beam_position,
                                                 normalize_image,
                                                 normalize_templates,
                                                )

In [11]:
mirrored = signs_n[0] == -1
x = putls.plot_template_over_pattern(image,
                                 simulations[indices_n[0]],
                                 in_plane_angle=angles_n[0],
                                 coordinate_system = "cartesian", 
                                 size_factor = 10,
                                 vmax=30, 
                                 max_r = 200,
                                 mirrored_template=mirrored,
                                 find_direct_beam=False,
                                 cmap = "viridis"
                                )

In [8]:
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls

In [9]:
def template_log_func(x):
    return np.log10(x + 0.01)

In [12]:
frac_keep = 0.8
n_best = 20
n_keep = None

delta_r = 0.5
delta_theta = 0.5
max_r = None
intensity_transform_function = template_log_func
direct_beam_position = None
normalize_image = True
normalize_templates = True

result, phasedict = iutls.index_dataset_with_template_rotation(dpau,
                                                    diff_lib,
                                                    phases = ["Au"],
                                                    n_best = n_best,
                                                    frac_keep = frac_keep,
                                                    n_keep = n_keep,
                                                    delta_r = delta_r,
                                                    delta_theta = delta_theta,
                                                    max_r = None,
                                                    intensity_transform_function=intensity_transform_function,
                                                    normalize_images = normalize_image,
                                                    normalize_templates=normalize_templates,
                                                    target='gpu'
                                                    )

[########################################] | 100% Completed |  7min  1.3s


In [2]:
from orix import plot
from orix.crystal_map import CrystalMap, Phase, PhaseList
from orix.io import load, save
from orix.quaternion import Orientation, Rotation, symmetry
from orix.vector import Vector3d
from pyxem.signals.indexation_results import results_dict_to_crystal_map

In [14]:
xmap = results_dict_to_crystal_map(result,phasedict)

In [14]:
xmap

Phase    Orientations  Name  Space group  Point group  Proper point group     Color
    0  16384 (100.0%)    Au         None         None                None  tab:blue
Properties: correlation, mirrored_template, template_index
Scan unit: px

In [4]:
xmap = load("Data/AuTwinTilt0xmapBGRED.h5") 

In [15]:
xmap.phases[0].space_group = 225

In [16]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.xvector())
rgb_Cux = ckey_m3m.orientation2color(xmap["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.yvector())
rgb_Cuy = ckey_m3m.orientation2color(xmap["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Cuz = ckey_m3m.orientation2color(xmap["Au"].orientations)

In [17]:
fig = plt.figure()
ax0 = fig.add_subplot(311, projection="plot_map")
ax1 = fig.add_subplot(312, projection="plot_map")
ax2 = fig.add_subplot(313, projection="plot_map")
ax0.set_title("X")
ax1.set_title("Y")
ax2.set_title("Z")
ax0.plot_map(xmap["Au"], rgb_Cux, scalebar = False)
ax1.plot_map(xmap["Au"], rgb_Cuy, scalebar = False)
ax2.plot_map(xmap["Au"], rgb_Cuz, scalebar = False)
ax0.tick_params(which = 'major', size = 0, labelsize = 0)
ax1.tick_params(which = 'major', size = 0, labelsize = 0)
ax2.tick_params(which = 'major', size = 0, labelsize = 0)

In [29]:
fig = plt.figure()
ax0 = fig.add_subplot(111, projection="plot_map")
ax0.set_title("Y")
ax0.plot_map(xmap["Au"], rgb_Cuy, scalebar = False)

In [68]:
xmap["Au"].plot(rgb_Cux, scalebar = False, remove_padding=True)

In [30]:
xmap['Au'].plot(value = xmap.correlation[:,0], scalebar = False, colorbar = True)

In [25]:
orifirst = xmap['Au'].orientations
store = np.zeros((xmap['Au'].size,15))
normcorrs = xmap['Au'].correlation[:,:]/np.linalg.norm(xmap['Au'].correlation[:,:]) 
for i in range (15):
    corr_weight = normcorrs[:,i]/normcorrs[:,0]
    # corr_weight = 1 # for unweighted, not recommened
    orinext = Orientation(xmap['Au'].rotations[:,i],symmetry=symmetry.Oh)
    store[:,i] = np.degrees((orinext - orifirst).angle.data) * abs(corr_weight)

mean_misori_weighted = np.mean(store[:,1::], axis = 1)

In [28]:
xmap['Au'].plot(value = mean_misori_weighted,scalebar = False, vmax = 10, colorbar = True)

In [55]:
normcorrs = xmap['Au'].correlation[:,:]/np.linalg.norm(xmap['Au'].correlation[:,:]) 
reliability = normcorrs[:,0] - normcorrs[:,1] 

In [56]:
xmap['Au'].plot(value = reliability,scalebar = False, colorbar = True)

In [29]:
n= np.linspace(0,4,num = 5)

array([0., 1., 2., 3., 4.])

In [33]:
xmap.orientations.scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [256]:
Orientation(xmap.rotations[0:3,0:10], symmetry=symmetry.Oh).scatter(projection= 'ipf', direction = Vector3d(((1, 0, 0),(0,1,0),(0,0,1))),c = n, cmap = 'viridis')

In [32]:
xmap.orientations.scatter(projection= 'axangle', cmap = 'viridis', c = n)

In [18]:
save(
    filename="Data/AuTwinTilt4xmapBGREDbetter.h5",
    object2write=xmap,
    overwrite=False,  # Default is False
)

In [3]:
xmap0 = load("Data/AuTwinTilt0xmapBGREDbetter.h5") 
xmap1 = load("Data/AuTwinTilt1xmapBGREDbetter.h5") 
xmap2 = load("Data/AuTwinTilt2xmapBGREDbetter.h5") 
xmap3 = load("Data/AuTwinTilt3xmapBGREDbetter.h5") 
xmap4 = load("Data/AuTwinTilt4xmapBGREDbetter.h5") 

In [118]:
simulations = diff_lib["Au"]['simulations']
#scan coordinate to check
px = 40
py = 57
# which solution to plot
n_sol = 0

index = np.ravel_multi_index((py,px),xmap.shape)
# query the necessary info from the solution
sim_sol_index = xmap.template_index[index,0]
mirrored_sol = xmap.mirrored_template[index,0]
in_plane_angle = np.rad2deg(xmap.rotations[index,0].to_euler())[0][0]
# # query the appropriate template
sim_sol = simulations[sim_sol_index]



fig = plt.figure()
ax0 = fig.add_subplot(121, projection="plot_map")
ax1 = fig.add_subplot(122)
ax0.set_title("Z")
ax0.plot_map(xmap['Au'],rgb_Auz, scalebar = False)
ax0.scatter(px,py, marker = 'X', c = 'g', s = 40)
putls.plot_template_over_pattern(dpau.inav[px, py].data,
                                 sim_sol,
                                 ax = ax1,
                                 in_plane_angle=in_plane_angle,
                                 coordinate_system = "cartesian", 
                                 size_factor = 10,
                                 vmax=30,
                                 max_r = 200,
                                 mirrored_template=mirrored_sol,
                                 find_direct_beam=False,
                                 cmap = "inferno",
                                 marker_color = "green"
                                )

(<AxesSubplot:>,
 <matplotlib.collections.PathCollection at 0x7f87291f1310>)

In [105]:
xmap = load("Data/AuTwinTilt1xmapBGREDbetter.h5") 

In [106]:
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.xvector())
rgb_Aux = ckey_m3m.orientation2color(xmap["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.yvector())
rgb_Auy = ckey_m3m.orientation2color(xmap["Au"].orientations)
ckey_m3m = plot.IPFColorKeyTSL(xmap.phases["Au"].point_group, direction=Vector3d.zvector())
rgb_Auz = ckey_m3m.orientation2color(xmap["Au"].orientations)

In [107]:
fig = plt.figure()
ax0 = fig.add_subplot(111, projection="plot_map")
ax0.set_title("Z")
ax0.plot_map(xmap["Au"], rgb_Auz, scalebar = False)

In [25]:
xmap['Au'].plot(value = xmap.correlation[:,0],scalebar = False, colorbar = True)

In [51]:
pos0 = (70,70)
pos1 = (65,40)
pos2 = (105,55)
pos3 = (101,63)
pos4 = (64,57)

In [80]:
twina = (8,89)
twinb = (8,93)

In [82]:
twinc = (98,80)
twind = (98,88)

In [88]:
twine = (28,94)
twinf = (26,87)

In [60]:
mis0 = xmap0['Au'].orientations - xmap0[pos0].orientations
mis1 = xmap1['Au'].orientations - xmap1[pos1].orientations
mis2 = xmap2['Au'].orientations - xmap2[pos2].orientations
mis3 = xmap3['Au'].orientations - xmap3[pos3].orientations
mis4 = xmap4['Au'].orientations - xmap4[pos4].orientations

mean0 = xmap0['Au'][np.rad2deg(mis0.angle.data) < 1].orientations.mean()
mean1 = xmap1['Au'][np.rad2deg(mis1.angle.data) < 1].orientations.mean()
mean2 = xmap2['Au'][np.rad2deg(mis2.angle.data) < 1].orientations.mean()
mean3 = xmap3['Au'][np.rad2deg(mis3.angle.data) < 1].orientations.mean()
mean4 = xmap4['Au'][np.rad2deg(mis4.angle.data) < 1].orientations.mean()

means = Orientation([mean0.data,mean1.data,mean2.data,mean3.data,mean4.data], symmetry=symmetry.Oh).reshape(5)
means.map_into_symmetry_reduced_zone()

print(np.rad2deg(means[0].angle_with(means[1]).data))
print(np.rad2deg(means[1].angle_with(means[2]).data))
print(np.rad2deg(means[0].angle_with(means[2]).data))
print(np.rad2deg(means[0].angle_with(means[3]).data))
print(np.rad2deg(means[3].angle_with(means[4]).data))
print(np.rad2deg(means[0].angle_with(means[4]).data))

[5.73980183]
[48.38864355]
[52.74756008]
[3.98274893]
[41.28816778]
[43.93057182]


In [89]:
np.rad2deg(xmap4[twine].orientations.angle_with(xmap4[twinf].orientations).data)

array([44.60049291])

In [112]:
np.rad2deg(xmap1[6,92].orientations.angle_with(xmap1[6,88].orientations).data)

array([44.52245587])

In [116]:
np.rad2deg(xmap1[57,40].orientations.angle_with(xmap1[53,41].orientations).data)

array([0.46701694])

In [53]:
np.rad2deg(xmap1[pos1].orientations.angle_with(xmap2[pos2].orientations).data)

array([45.25396447])

In [76]:
means.scatter(projection='ipf', c = ('g','b','b','r','r'))

In [197]:
newbest = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]),np.int16)

for k in range(20):
    newresult= np.zeros((1,5,3))
    for i in range(result['orientation'].shape[0]):
        for j in range(result['orientation'].shape[1]):
            newresult[i,j] = result['orientation'][i,j,newbest[i,j],:]

    misoris = np.zeros((result['orientation'].shape[0],result['orientation'].shape[1]-1))

    oris = Orientation.from_euler(np.radians(newresult[:,:,:]),symmetry=symmetry.Oh)
    mis = (oris[:,0] - oris[:,1])
    misoris[:,0] = np.degrees((mis).angle.data)
    mis = (oris[:,1] - oris[:,2])
    misoris[:,1] = np.degrees((mis).angle.data)
    mis = (oris[:,0] - oris[:,3])
    misoris[:,2] = np.degrees((mis).angle.data)
    mis = (oris[:,3] - oris[:,4])
    misoris[:,3] = np.degrees((mis).angle.data)
    
    misoris[:,3]

In [196]:
Orientation.from_euler(np.radians(newresult[:,:,:]),symmetry=symmetry.Oh)[:]

Orientation (1, 5) m-3m
[[[ 0.1644  0.3728  0.0878  0.909 ]
  [ 0.914  -0.0309  0.3586 -0.1873]
  [ 0.9167 -0.0108  0.3419 -0.2064]
  [ 0.1586  0.3331  0.1003  0.924 ]
  [ 0.1559  0.3051  0.1141  0.9325]]]